In [ ]:
import argparse
import os, sys
import json
import numpy as np
import time

import WhiskiWrap as ww
from WhiskiWrap import load_whisker_data as lwd
# add directory above notebook to path
sys.path.append(os.path.dirname(os.path.abspath('')))
import whiskerpad as wp
# Check that whisk binaries are executables and update permissions if necessary
from wwutils.whisk_permissions import update_permissions
update_permissions()

In [ ]:
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
input_dir=Path('/data')
# list all the files in the data directory
# data_files = list(input_dir.glob('*'))
# print(data_files)
video_name='sc016_0630_001_30sWhisking.mp4'
base_name='sc016_0630_001'
input_file=input_dir/video_name

# load the video's first frame 
frame_num = 0
cap = cv2.VideoCapture(str(input_file))
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
ret, frame = cap.read()
cap.release()
# display the image,
# plt.imshow(frame)
# plt.axis('off')
# plt.show()

# # Save the image as base_name_first_frame.png
# output_file = input_dir / f'{base_name}_first_frame.png'
# cv2.imwrite(str(output_file), frame)

#### Run whisker tracking on the first frame

In [ ]:
splitUp=True
output_dir = input_dir / 'frame_0_whiskers'
nproc=40

In [ ]:
# whiskerpad=wp.Params(str(input_file), splitUp, base_name)
# whiskerpadParams, splitUp = wp.WhiskerPad.get_whiskerpad_params(whiskerpad)

image_halves, image_side, face_side, fp = wp.get_side_image(str(input_file), splitUp)
# print each image halvse side by side
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(image_halves[0], cmap='gray')
ax[0].axis('off')
ax[0].set_title(f"Face side: {face_side[0]}")
ax[1].imshow(image_halves[1], cmap='gray')
ax[1].axis('off')
ax[1].set_title(f"Face side: {face_side[1]}")
plt.show()

# Save each side as tif
output_dir = input_dir / 'frame_0_whiskers'
output_dir.mkdir(exist_ok=True)
for i, side in enumerate(face_side):
    output_file = output_dir / f'{base_name}_first_frame_{side}.tif'
    cv2.imwrite(str(output_file), image_halves[i])

In [ ]:
# Load whiskerpad json file
whiskerpad_file = os.path.join(input_dir, f'whiskerpad_{base_name}.json')
# whiskerpad_file = os.path.join(input_dir, f'whiskerpad_{os.path.basename(input_file).split(".")[0]}.json')

if not os.path.exists(whiskerpad_file):
# If whiskerpad file does not exist, create it
    print('Creating whiskerpad parameters file.')
    whiskerpad=wp.Params(input_file, splitUp, base_name)
    # Get whiskerpad parameters
    whiskerpadParams, splitUp = wp.WhiskerPad.get_whiskerpad_params(whiskerpad)
    # Save whisking parameters to json file
    wp.WhiskerPad.save_whiskerpad_params(whiskerpad, whiskerpadParams)

with open(whiskerpad_file, 'r') as f:
    whiskerpad_params = json.load(f)

# Check that left and right whiskerpad parameters are defined
if np.size(whiskerpad_params['whiskerpads'])<2:
    raise Exception('Missing whiskerpad parameters in whiskerpad json file.')

# Get side types (left / right or top / bottom)
side_types = [whiskerpad['FaceSide'].lower() for whiskerpad in whiskerpad_params['whiskerpads']]

In [ ]:
#######################
### Run whisker tracking
########################

for side in side_types:
    print(f'Running whisker tracking for {side} face side video')

    # Time the tracking
    start_time_track = time.time()

    image_filename = output_dir / f'{base_name}_first_frame_{side}.tif'
    result_dict = ww.trace_and_measure_chunk(image_filename,
                                            delete_when_done=False,
                                            face=side,
                                            classify={'px2mm': '0.04', 'n_whiskers': '3'})
    

    time_track = time.time() - start_time_track
    print(f'Tracking took {time_track} seconds.')

In [ ]:
# Load the whisker data and plot whiskers on each image
